In [ ]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
import os
import time
from sklearn.neural_network import MLPRegressor
from envs.custom_env_dir.sup_model import SupModel

if __name__ == '__main__':

    cwd = os.getcwd()
    
    ''' THESE FILES ARE NOT INCLUDED IN THE GIT-REPOSITORY DUE TO THEIR SIZE '''
    ''' YOU CAN COMPILE THESE YOURSELF USING THE main_benchmark.ipynb NOTEBOOK '''
    ''' SELECT EITHER supervised_training_set=True OR supervised_test_set=True  '''
    train_data = cwd + '/data/Imitation_train_sup-dataset.csv'
    test_data = cwd + '/data/Imitation_test_sup-dataset.csv'
    
    
    ''' CHOOSE INPUT FEATURES BY SPECIFYING BOTH obs-STRING AND RESPECTIVE in_col-LIST '''
    #obs = 'obs1(t_step)'
    #in_col = ['t_step']
    #obs = 'obs2(t_step,daycat)'
    #in_col = ['t_step','day_cat']
    #obs = 'obs3(t_sin,t_cos,is_we)'
    #in_col = ['t_sin','t_cos','is_we']
    obs = 'obs2(t_step,daycat)'
    in_col = ['t_step','day_cat']
    #obs = 'obs3(t_sin,t_cos,daycat)''
    #in_col = ['t_sin','t_cos','day_cat']
    #obs = 'obs4(t_sin,t_cos,daycat,temp0)'
    #in_col = ['t_sin','t_cos','day_cat','temp']

    # Specify target column
    out_col = ['env_action']
    
    ''' SELECT IF WHICH MODEL YOU WANT TO TRAIN '''
    k_neighbors = False
    mlp = True
    
    
    ''' NO ADJUSTMENTS REQUIRED FROM HERE ''' 
    # load dataset
    x_train, y_train, x_test, y_test = SupModel().load_data(train_data, test_data, in_col, out_col)
    
    
    if k_neighbors:
        # ------------------------------ k-nearest neighbors
        for k in ([15]):

            print('k-nearest neighbors will be trained with k =', k)

            filename = 'KNeighbors_k(' + str(k) + ')' 
            store_dir = cwd +'/knn_models/'+ filename + '_' + obs 

            # Train and store model
            start = time.time()
            SupModel().train_model_kneighbors(store_dir, k, x_train, y_train, x_test, y_test)
            end = time.time()
            print('This took ', end - start, ' seconds.')

            # Load model
            model = SupModel().load_model_kneighbors(store_dir, k)

            # Test model
            SupModel().test_model(model, store_dir, x_test, y_test)
                   
    if mlp:
        # ------------------------------ MLP Classifier
        for hl in ([(16)]):            # (16,16)
            for af in (['relu']):      # ‘identity’, ‘logistic’, ‘tanh’, ‘relu’
                for sl in (['adam']):  # ‘lbfgs’, ‘sgd’, ‘adam’

                    print('MLP classifier will be trained with hl: ', hl, ' af: ', af, ' sl: ', sl)

                    filename = 'MLP_hl(' + str(hl) + ')_af(' + str(af) + ')_sl(' + str(sl) + ')'
                    store_dir = cwd +'/mlp_models/'+ filename + '_' + obs            

                    # Train and store model
                    start = time.time()
                    SupModel().train_model_mlp(store_dir, hl, af, sl, x_train, y_train, x_test, y_test)
                    end = time.time()
                    print('This took ', end - start, ' seconds.')

                    # Load model
                    model = SupModel().load_model_mlp(store_dir, hl, af, sl)

                    # Test model
                    SupModel().test_model(model, store_dir, x_test, y_test)